In [1]:
from pathlib import Path
import sys

sys.path.append('..')
from config import work_dir, bounds
from src.processing_utils import read_products, insar_stack_overview

INFO: org.esa.snap.core.gpf.operators.tooladapter.ToolAdapterIO: Initializing external tool adapters
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Incompatible GDAL 3.6.2 found on system. Internal GDAL 3.0.0 from distribution will be used.
INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.
INFO: org.esa.snap.core.util.EngineVersionCheckActivator: Please check regularly for new updates for the best SNAP experience.
Currently installed 8.0, available is 9.0.0.
Please visit http://step.esa.int



In [3]:
# Specifying the Raw Product Directory
raw_dir = Path(work_dir, 'data_raw')

In [4]:
# Getting all .zip files in the Raw Product Directory and sorting by date
raw_paths = [(f, f.stem[17:25]) for f in Path(raw_dir).glob("*.zip")]

if len(raw_paths) >= 2:
    raw_paths_sorted = sorted(raw_paths, key=lambda x: x[1])
else:
    raise(ValueError('Insufficient Number of Files (Minimum of 2)'))

ValueError: Insufficient Number of Files (Minimum of 2)

# InSAR Baseline Information

In [6]:
for path_1, path_2 in zip(raw_paths_sorted, raw_paths_sorted[1:]):
    read_1, read_2 = read_products(path_1[0], path_2[0])
    perpendicular_baseline, temporal_baseline = insar_stack_overview(read_1, read_2)

    if abs(perpendicular_baseline) < 150 or temporal_baseline < 6 or temporal_baseline > 12:
        result = 'FAIL'
    else:
        result = 'PASS'

    print(f'{path_1[0].name},{path_2[0].name},{perpendicular_baseline},{temporal_baseline},{result}\n')

S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip,S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip,-66.79852294921875,12.00001049041748,FAIL

S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip,S1B_IW_SLC__1SDV_20200609T213952_20200609T214020_021962_029AEE_986D.zip,34.98725509643555,12.000008583068848,FAIL



INFO: org.esa.s2tbx.dataio.gdal.GDALVersion: Internal GDAL 3.0.0 set to be used by SNAP.


# TOPS Analyser

In [17]:
import stsa
import geopandas as gpd
from shapely.geometry import Polygon

In [18]:
aoi_polygon = Polygon([(bounds[0], bounds[1]), (bounds[2], bounds[1]), (bounds[2], bounds[3]), (bounds[0], bounds[3])])
aoi_gdf = gpd.GeoDataFrame(geometry=[aoi_polygon])
aoi_gdf.crs = "EPSG:4326"

s1 = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'], polarization='vv')

In [19]:
for path_1, path_2 in zip(raw_paths_sorted, raw_paths_sorted[1:]):
    # Subswath and Burst Selection for read_1
    s1.load_zip(zip_path=str(path_1[0]))
    s1_results_1 = s1.intersecting_bursts(aoi_gdf)

    # Subswath and Burst Selection for read_2
    s1.load_zip(zip_path=str(path_2[0]))
    s1_results_2 = s1.intersecting_bursts(aoi_gdf)

    # Sorting the Results
    subswaths = list({x[0] for x in s1_results_1})

    subswath_bursts_1 = []
    subswath_bursts_2 = []
    for subswath in subswaths:
        subswath_bursts_1.append(
            (
                subswath,
                min(x[1] for x in s1_results_1 if x[0] == subswath),
                max(x[1] for x in s1_results_1 if x[0] == subswath),
            )
        )
        subswath_bursts_2.append(
            (
                subswath,
                min(x[1] for x in s1_results_2 if x[0] == subswath),
                max(x[1] for x in s1_results_2 if x[0] == subswath),
            )
        )

    # Compiling the Results
    subswaths_dict = {i:{} for i in subswaths}
    for subswath in subswaths:
        for subswath_bursts in subswath_bursts_1:
            if subswath_bursts[0] == subswath:
                subswaths_dict[subswath]['product_1'] = (subswath_bursts[1], subswath_bursts[2])
        for subswath_bursts in subswath_bursts_2:
            if subswath_bursts[0] == subswath:
                subswaths_dict[subswath]['product_2'] = (subswath_bursts[1], subswath_bursts[2])
    
    # Printing the Results
    for subswath, bursts in subswaths_dict.items():
        product_1_first = bursts['product_1'][0]
        product_1_last = bursts['product_1'][1]
        product_2_first = bursts['product_2'][0]
        product_2_last = bursts['product_2'][0]

        print(f'Subswath: {subswath}')
        print(f'Product 1 Bursts: {product_1_first}, {product_1_last}')
        print(f'Product 2 Bursts: {product_2_first}, {product_2_last}')
        print('\n')

Loaded ZIP file: S1B_IW_SLC__1SDV_20200516T213951_20200516T214019_021612_029081_ACBD.zip
Loaded location grid with 10 bursts and 231 coordinates
Found 6 XML paths
Loaded ZIP file: S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip
Found 6 XML paths
Subswath: IW2
Product 1 Bursts: 6, 7
Product 2 Bursts: 6, 6


Subswath: IW1
Product 1 Bursts: 6, 8
Product 2 Bursts: 6, 6


Loaded ZIP file: S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip
Found 6 XML paths
Loaded ZIP file: S1B_IW_SLC__1SDV_20200609T213952_20200609T214020_021962_029AEE_986D.zip
Found 6 XML paths
Subswath: IW2
Product 1 Bursts: 6, 7
Product 2 Bursts: 6, 6


Subswath: IW1
Product 1 Bursts: 6, 8
Product 2 Bursts: 6, 6




# Visualising via Web Map

In [20]:
s1.load_zip(zip_path=str(path_1[0]))
s1.visualize_webmap(polygon=aoi_gdf)

Loaded ZIP file: S1B_IW_SLC__1SDV_20200528T213952_20200528T214019_021787_0295B1_17C1.zip
Found 6 XML paths


In [22]:
s1.load_zip(zip_path=str(path_2[0]))
s1.visualize_webmap(polygon=aoi_gdf)

Loaded ZIP file: S1B_IW_SLC__1SDV_20200609T213952_20200609T214020_021962_029AEE_986D.zip
Found 6 XML paths
